In [ ]:
from skopt.space import Real
from pssr.crappifiers import AdditiveGaussian, Poisson
from pssr.data import ImageDataset
from pssr.train import approximate_crappifier

In [ ]:
gaussian_crappifier = AdditiveGaussian(mean=0, deviation=5)
poisson_crappifier = Poisson(intensity=1.5)

dataset = ImageDataset("data/EM_hr_1_10", crappifier=poisson_crappifier)

In [ ]:
gaussian_space = [Real(-3, 3),
         Real(0, 10),
]

poisson_space = Real(0, 3)

In [ ]:
result = approximate_crappifier(Poisson, poisson_space, dataset, n_samples=10, opt_kwargs=dict(n_calls=100, n_initial_points=10))